# Prepare

In [ ]:
!nvidia-smi

In [ ]:
import torch
from functorch import vmap
x = torch.randn(3)
y = vmap(torch.sin)(x)
assert torch.allclose(y, x.sin())

In [ ]:
import functorch
!python -m xformers.info

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from torch import autocast

In [ ]:
# remote model path
# model_id = "stabilityai/stable-diffusion-2-base"

# local model path
model_id = "models/stable-diffusion-v2-512x512"

# Use the Euler scheduler here instead
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)
# pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to(0)
pipe.enable_attention_slicing()
pipe.device

In [ ]:
# save the model locally after initial download
# pipe.save_pretrained('models/stable-diffusion-v2-512x512')

In [ ]:
!pip install python-slugify

# RUN

In [ ]:
from slugify import slugify
from time import time

prompt = "low angle 50mm photograph of a majestic bacon-cheesehamburger composed of grilled beef patties and fresh salad and tomato and secret sauce and melted american cheese and sesame seed buns, 4k award winning food photography, trending on artstation"
neg_prompt = "blurry, out of focus"

set_guidance_scale = 7.5

def txt2img(prompt, neg_prompt, n_iterations, display_image):
    
    seed_generator = torch.Generator('cuda').manual_seed(42)
    image = None
    for i in range(n_iterations):
        with autocast('cuda'):
          image = pipe(prompt, negative_prompt=neg_prompt, height=512, width=512,generator=seed_generator, guidance_scale = set_guidance_scale).images[0]
          image.save(f"images/{slugify(prompt)[:100]}_{i}_{time()}.png")
        if display_image:
            display(image)

    # return image

In [ ]:
image = txt2img(prompt,neg_prompt, 50, True)